In [0]:
import os
import requests
from IPython.display import clear_output

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import json
from pandas.io.json import json_normalize

import cv2
import torch

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#Set path to your own Google Drive before running
# path = '/gdrive/My Drive/NYU/Spring 2019/Urban Science Intensive I/Getting to Zero Capstone /VML'
path = '/gdrive/My Drive/Getting to Zero Capstone /VML'

os.listdir(path)

FileNotFoundError: ignored

## Data processing

### Crosswalks

In [0]:
def load_dataturks_json(json_path):
  
  data = []
  with open(json_path) as f:  
      for line in f:
          data.append(json.loads(line))

  df = json_normalize(data)
  df['image_id'] = df.index + 1

  return df


def extract_annotations(df):
  
  data = []
  for i, row in df.iterrows():

    #Add -1 if no annotations for this image
    if row.annotation is None:
      annotation = {
            'image_id': row.image_id,
            'type' : None,
            'x1' : -1,
            'x2' : -1,
            'y1' : -1,
            'y2' : -1
                     }
      data.append(annotation)
      
    else:
      
      #Double loop: each box can have more than one associated label
      for a in row.annotation:
        for label in a['label']:
          
          #Need to check if this is the right way to calculate box coords
          annotation = {
              'image_id': row.image_id,
              'type' : label,
              'x1' : a['imageWidth']*min(a['points'], key=lambda x: x[0])[0],
              'x2' : a['imageWidth']*max(a['points'], key=lambda x: x[0])[0],
              'y1' : a['imageHeight']*min(a['points'], key=lambda x: x[1])[1],
              'y2' : a['imageHeight']*max(a['points'], key=lambda x: x[1])[1]
                     }

        data.append(annotation)

  return json_normalize(data)

def extract_annotations_reverse(df):  
  data = []
  for i, row in df.iterrows():
    #Add -1 if no annotations for this image
    if row.annotation is None:
      annotation = {
            'image_id': row.image_id,
            'type' : None,
            'x1' : -1,
            'x2' : -1,
            'y1' : -1,
            'y2' : -1
                     }
      data.append(annotation)      
    else:      
      #Double loop: each box can have more than one associated label
      for a in row.annotation:
        for label in a['label']:          
          #Need to check if this is the right way to calculate box coords
          annotation = {
              'image_id': row.image_id,
              'type' : label,
              'x1' : a['imageWidth']*(1-(max(a['points'], key=lambda x: x[0])[0])),
              'x2' : a['imageWidth']*(1-(min(a['points'], key=lambda x: x[0])[0])),
              'y1' : a['imageHeight']*min(a['points'], key=lambda x: x[1])[1],
              'y2' : a['imageHeight']*max(a['points'], key=lambda x: x[1])[1]
                     }
        data.append(annotation)
  return json_normalize(data)


def download_images(df, path):
  
  #Create path if it doesn't exist
  if not os.path.isdir(path):
    os.mkdir(path)
  
  for i, row in df.iterrows():
    
    print ('Downloading image', row.image_id, 'from', row.content)
    
    #Get image extension
    ext = row.content.split('.')[-1]
    
    #Download image
    image_req = requests.get(row.content)
    
    if image_req.status_code == 200:
      
      #Save to file
      with open(path + '/' + str(row.image_id) + '.' + ext, 'wb') as f:
        f.write(image_req.content)
        
    clear_output(wait=True)
  
  print('Done!')
  
def flip_images(path):
  
  directory = os.fsencode(path)

  for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if (filename.endswith(".jpg")) and ('M' not in filename): 
        imageSource = os.path.join(os.fsdecode(directory), filename)
        # Add suffix m to output
        imageExport = imageSource.split('.')[0]+'M'+'.jpg'
        img = cv2.imread(imageSource)
        horizontal_img = img.copy()
        # use cv2 flip with '+1' parameter to flip horizontally
        horizontal_img = cv2.flip(img, +1)
        # export flipped image to same folder
        cv2.imwrite(imageExport, horizontal_img)
     else:
         continue
        
  print('Done!')

In [0]:
#Load crosswalks json
crosswalks_df = load_dataturks_json(path + '/NYU_capstone_VML/Crosswalks.json')
crosswalks_df.head()

FileNotFoundError: ignored

In [0]:
#Extract annotations into a separate df with image id
crosswalks_output_df = extract_annotations(crosswalks_df)
crosswalks_output_df.head(10)

,image_id,type,x1,x2,y1,y2
0,1,None,-1.000000,-1.000000,-1.000000,-1.000000
1,2,cmother,861.217941,1280.000000,732.325876,924.561418
2,3,None,-1.000000,-1.000000,-1.000000,-1.000000
3,4,None,-1.000000,-1.000000,-1.000000,-1.000000
4,5,None,-1.000000,-1.000000,-1.000000,-1.000000
5,6,whitline,365.714286,476.458753,616.463476,714.478589
6,6,whitline,417.223340,1102.293763,688.685139,768.644836
7,7,None,-1.000000,-1.000000,-1.000000,-1.000000
8,8,None,-1.000000,-1.000000,-1.000000,-1.000000
9,9,None,-1.000000,-1.000000,-1.000000,-1.000000


In [0]:
crosswalks_output_df.to_csv(path + '/C_Output.csv', index=False)

In [0]:
#Download images
download_images(crosswalks_df, path + '/Crosswalks')

Done!


### Sidewalks

In [0]:
#Load crosswalks json
sidewalks_df = load_dataturks_json(path + '/Dataturks_files/Sidewalks.json')
sidewalks_df.head()

,annotation,content,extras,metadata.evaluation,metadata.first_done_at,metadata.last_updated_at,metadata.last_updated_by,metadata.sec_taken,metadata.status,image_id
0,"[{'label': ['sidewalk'], 'shape': 'rectangle',...",http://com.dataturks.a96-i23.open.s3.amazonaws...,None,NONE,1553007948000,1553007948000,wbgnJzNT6qRX9iHzCZtmmdrcaHz1,0,done,1
1,"[{'label': ['sidewalk'], 'shape': 'rectangle',...",http://com.dataturks.a96-i23.open.s3.amazonaws...,None,NONE,1553712610000,1553712610000,GNZBHtCvCfcyQcuSAAS2xwweDvI2,0,done,2
2,"[{'label': ['sidewalk'], 'shape': 'rectangle',...",http://com.dataturks.a96-i23.open.s3.amazonaws...,None,NONE,1553014154000,1553014154000,wbgnJzNT6qRX9iHzCZtmmdrcaHz1,0,done,3
3,"[{'label': ['sidewalk'], 'shape': 'rectangle',...",http://com.dataturks.a96-i23.open.s3.amazonaws...,None,NONE,1553621911000,1553621911000,Q5KvZ1Uti4SZ83gzxPRXRAgSswF3,0,done,4
4,"[{'label': ['sidewalk'], 'shape': 'rectangle',...",http://com.dataturks.a96-i23.open.s3.amazonaws...,None,NONE,1553621070000,1553621070000,Q5KvZ1Uti4SZ83gzxPRXRAgSswF3,0,done,5


In [0]:
#Extract annotations into a separate df with image id
sidewalks_output_df = extract_annotations(sidewalks_df)
sidewalks_output_df.head(10)

,image_id,type,x1,x2,y1,y2
0,1,sidewalk,33.530752,1271.252847,747.243243,837.553343
1,2,sidewalk,6.030624,363.345112,690.709867,769.131075
2,2,sidewalk,818.657244,1270.954064,671.104566,889.779087
3,3,sidewalk,858.734177,1278.527043,702.802878,860.454676
4,3,sidewalk,8.837745,486.075949,699.856115,914.969784
5,3,sdwkshde,17.675489,309.321059,798.572662,873.715108
6,3,street_trees,8.837745,268.078251,511.263309,810.359712
7,4,sidewalk,668.029197,1270.656934,725.430524,898.041002
8,4,street_trees,684.379562,1200.583942,384.874715,676.446469
9,4,street_trees,14.014599,334.014599,310.232346,685.776765


In [0]:
sidewalks_output_df.to_csv(path + '/S_Output.csv', index=False)

In [0]:
#Download images
download_images(sidewalks_df, path + '/Sidewalks')

Done!


## Data augmentation

In [0]:
# flip Crosswalk images horizontally
flip_images(path+"/Crosswalks")

Done!


In [0]:
# flip Sidewalk images horizontally
flip_images(path+"/Sidewalks")

Done!


In [0]:
#Extract annotations reversed into a separate df with image id
sidewalks_reverse_output_df = extract_annotations_reverse(sidewalks_df)
sidewalks_reverse_output_df.head(10)

,image_id,type,x1,x2,y1,y2
0,1,sidewalk,8.747153,1246.469248,747.243243,837.553343
1,2,sidewalk,916.654888,1273.969376,690.709867,769.131075
2,2,sidewalk,9.045936,461.342756,671.104566,889.779087
3,3,sidewalk,1.472957,421.265823,702.802878,860.454676
4,3,sidewalk,793.924051,1271.162255,699.856115,914.969784
5,3,sdwkshde,970.678941,1262.324511,798.572662,873.715108
6,3,street_trees,1011.921749,1271.162255,511.263309,810.359712
7,4,sidewalk,9.343066,611.970803,725.430524,898.041002
8,4,street_trees,79.416058,595.620438,384.874715,676.446469
9,4,street_trees,945.985401,1265.985401,310.232346,685.776765


In [0]:
#Extract annotations reversed into a separate df with image id
crosswalks_reverse_output_df = extract_annotations_reverse(crosswalks_df)
crosswalks_reverse_output_df.head(10)

,image_id,type,x1,x2,y1,y2
0,1,None,-1.000000,-1.000000,-1.000000,-1.000000
1,2,cmother,0.000000,418.782059,732.325876,924.561418
2,3,None,-1.000000,-1.000000,-1.000000,-1.000000
3,4,None,-1.000000,-1.000000,-1.000000,-1.000000
4,5,None,-1.000000,-1.000000,-1.000000,-1.000000
5,6,whitline,803.541247,914.285714,616.463476,714.478589
6,6,whitline,177.706237,862.776660,688.685139,768.644836
7,7,None,-1.000000,-1.000000,-1.000000,-1.000000
8,8,None,-1.000000,-1.000000,-1.000000,-1.000000
9,9,None,-1.000000,-1.000000,-1.000000,-1.000000


## Data Augamentation2: Rotation

# 

In [0]:
import argparse
import imutils

def rotate_images(path, angle):
  #‘’‘angle: rotation angle could be 45, 90, 135, 180, 225, 270, 315 clockwise’‘’
  
  directory = os.fsencode(path)

  for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if (filename.endswith(".jpg") and ('M' not in filename) and \
         ('R' not in filename)): 
        imageSource = os.path.join(os.fsdecode(directory), filename)
        # Add suffix m to output
        imageExport = imageSource.split('.')[0]+'R'+ str(angle) + '.jpg'
        img = cv2.imread(imageSource)
        rotation_img = img.copy()
        
        #use rotate_bound to make sure the rotated image is not cut off the edge by original size
        rotation_img = imutils.rotate_bound(rotation_img, angle)
 
        # export flipped image to same folder
        cv2.imwrite(imageExport, rotation_img)
     else:
         continue
        
  print('Done!')

In [0]:
#Haven't run this code in this Notebook, the rotated coordination haven't been calcuated
for i in [5,10,350,355]:
  rotate_images(path+"/Crosswalks",i)
  rotate_images(path+"/Sidewalks",i)
  

In [0]:


def rotate_coordinate(src, xmin, ymin, xmax, ymax, angle, scale=1.):
    w = src.shape[1]
    h = src.shape[0]
    rangle = np.deg2rad(angle)  # angle in radians (pie)
    # now calculate new image width and height
    # get the rotated width ane height
    nw = (abs(np.sin(rangle)*h) + abs(np.cos(rangle)*w))*scale
    nh = (abs(np.cos(rangle)*h) + abs(np.sin(rangle)*w))*scale
    # ask OpenCV for the rotation matrix
    rot_mat = cv2.getRotationMatrix2D((nw*0.5, nh*0.5), angle, scale)
    #print(rot_mat)
    # calculate the move from the old center to the new center combined
    # with the rotation
    rot_move = np.dot(rot_mat, np.array([(nw-w)*0.5, (nh-h)*0.5,0]))
    #print(np.array([(nw-w)*0.5, (nh-h)*0.5,0]))
    # the move only affects the translation, so update the translation
    # part of the transform
    rot_mat[0,2] += rot_move[0]
    rot_mat[1,2] += rot_move[1]
    # rot_mat is the final rotated matrix
    # get 4 original matrix middle points，and rotated the 4 points
    point1 = np.dot(rot_mat, np.array([w*(xmin+xmax)/2, h*ymin, 1]))
    point2 = np.dot(rot_mat, np.array([w*xmax, h*(ymin+ymax)/2, 1]))
    point3 = np.dot(rot_mat, np.array([w*(xmin+xmax)/2, h*ymax, 1]))
    point4 = np.dot(rot_mat, np.array([w*xmin, h*(ymin+ymax)/2, 1]))
    # concat np.array
    concat = np.vstack((point1, point2, point3, point4))
    # change array type
    concat = concat.astype(np.int32)
    #print (concat)
    rx, ry, rw, rh = cv2.boundingRect(concat)
    #print(rx,ry,rw,rh)
    return rx, ry, rw, rh

In [0]:
def extract_annotations_rotate_test(df,angle):  
  #angle number should be matched with rotate angle: .
  data = []
  for i, row in df.iterrows():
    
    #Add -1 if no annotations for this image
    if row.annotation is None:
      annotation = {
            'image_id': row.image_id,
            'rotate_angle': angle,
            'type' : None,
            'x1' : -1,
            'x2' : -1,
            'y1' : -1,
            'y2' : -1
                     }
      data.append(annotation) 
    else:      
      #Double loop: each box can have more than one associated label
      filename = str(row.image_id) +'R'+str(angle)+'.jpg'
      imageSource = os.path.join(os.fsdecode(path+"/Crosswalks"), filename)
      scr = cv2.imread(imageSource)
      #prepare to get the original point coordination
      
      for a in row.annotation:
        (new_h, new_w) = (a['imageHeight'], a['imageWidth'])
        for label in a['label']:  
          #get label point coordination  
          old_x1 = (min(a['points'], key=lambda x: x[0])[0])
          old_x2 = (max(a['points'], key=lambda x: x[0])[0])
          old_y1 = (min(a['points'], key=lambda x: x[1])[1])
          old_y2 = (max(a['points'], key=lambda x: x[1])[1])
          #new coordination
          nx,ny,nw,nh = rotate_coordinate(scr,old_x1,old_y1,old_x2,old_y2,angle,scale =1.)
          #new coordination between [0,1]
          nx1 = nx/new_w
          ny1 = ny/new_h
          nx2 = (nw+ nx)/new_w
          ny2 = (nh + ny)/new_h
          
        
          #Need to check if this is the right way to calculate box coords
          annotation = {
              'image_id': row.image_id,
              'rotate_angle': angle,
              'type' : label,
              'x1' : nx1 ,
              'x2' : nx2,
              'y1' : ny1,
              'y2' : ny2
                     }
        data.append(annotation)
        
      
        
        
        
  return json_normalize(data)

In [0]:
#haven't run
for i in [5,10,350,355]:
  crosswalks_rotate_output = extract_annotations_rotate_test(crosswalks_df,i)
  crosswalks_rotate_output.to_csv(path + '/C_rotate'+str(i)+'_Output.csv', index=False)
  sidewalks_rotate_output =extract_annotations_rotate_test(sidewalks_df,i)
  sidewalks_rotate_output.to_csv(path + '/S_rotate'+str(i)+'_Output.csv', index=False)
  